In [1]:
import pandas as pd
import os
from os.path import isfile, join

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import numpy as np

In [2]:
# show all rows
pd.options.display.max_rows = 999

In [60]:
# read file from data folder
datafolder = 'data/'
datafiles = [f for f in os.listdir(datafolder) if isfile(join(datafolder, f))]
for file in datafiles:
    datatable = pd.ExcelFile(os.path.join(datafolder,file))
    datatable_pd = datatable.parse(datatable.sheet_names[0])

In [4]:
# number of features and samples
datatable_pd.shape

(248, 117)

In [5]:
# display data
datatable_pd.head()

,CHROM,POS,ID,REF,ALT,INFO,Type,Length,isTv,isDerived,...,Intron,oAA,nAA,Grantham,PolyPhenCat,PolyPhenVal,SIFTcat,SIFTval,RawScore,PHRED
0,18,47352774,MYO5B:c.5616-2A>G,T,C,Pathogenic,SNV,0,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334537,6.023
1,18,47361716,MYO5B:c.5392C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,Q,*,NaN,NaN,NaN,NaN,NaN,14.849415,48.000
2,18,47361725,MYO5B:c.5383C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,R,*,NaN,NaN,NaN,NaN,NaN,15.701726,52.000
3,18,47365526,MYO5B:c.4840C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,Q,*,NaN,NaN,NaN,NaN,NaN,14.577744,46.000
4,18,47365610,MYO5B:c.4755_4756dupT,C,CCA,Pathogenic,INS,2,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.553096,35.000


In [6]:
# display the description for the numerical columns
datatable_num = datatable_pd.select_dtypes(exclude=[np.object])
# exclude the CHROM, POS
des_num = datatable_num[datatable_num.columns.difference(['CHROM', 'POS'])].describe().T
# drop features without any available samples
des_num = des_num.drop(des_num[des_num['count'] < 1].index)
des_num.sort_values(by='count').round(2)

,count,mean,std,min,25%,50%,75%,max
tOverlapMotifs,7.0,4.00,2.24,1.00,2.50,4.00,6.00,6.00
motifDist,7.0,-0.13,0.65,-1.32,-0.39,0.00,0.32,0.53
mirSVR-Aln,18.0,141.44,15.37,120.00,123.25,147.00,154.25,161.00
mirSVR-E,18.0,-12.33,5.04,-24.38,-15.14,-11.92,-9.63,-4.41
mirSVR-Score,18.0,-0.27,0.31,-1.09,-0.31,-0.26,-0.04,-0.01
scoreSegDup,19.0,0.93,0.00,0.93,0.93,0.93,0.93,0.93
TFBS,21.0,3.43,2.93,1.00,1.00,3.00,3.00,11.00
TFBSPeaks,21.0,4.05,3.47,1.00,1.00,3.00,5.00,11.00
TFBSPeaksMax,21.0,57.83,27.67,25.16,36.81,45.93,63.90,105.54
EncOCpolIISig,28.0,0.01,0.02,0.00,0.00,0.00,0.01,0.06


In [7]:
# display the description for columns with object type of values in the table
datatable_obj = datatable_pd.select_dtypes(include=[np.object])
des_obj = datatable_obj.describe()
nr_unique = []
unique =[]
for col_name in datatable_obj.columns:
    nr_unique.append(len(datatable_obj[col_name].unique()))
    unique.append(datatable_obj[col_name].unique())
assert len(nr_unique) == len(des_obj.columns) == len(unique)
des_obj = des_obj.T
des_obj['nr_unique'] = nr_unique
des_obj['unique'] = unique
des_obj

,count,unique,top,freq,nr_unique
ID,248,"[MYO5B:c.5616-2A>G, MYO5B:c.5392C>T, MYO5B:c.5...",PopulationGAVINr0.4,146,103
REF,248,"[T, G, C, AA, A, CGGG, TC, AG, TCTGA, CA, TA, ...",C,75,20
ALT,248,"[C, A, CCA, GC, G, AGAG, CC, T, TC, TGAG, TTCT...",A,77,14
INFO,248,"[Pathogenic, Benign]",Benign,195,2
Type,248,"[SNV, INS, DEL]",SNV,219,3
isTv,218,"[False, nan, True]",False,134,3
AnnoType,248,"[Transcript, CodingTranscript, RegulatoryFeature]",CodingTranscript,189,3
Consequence,248,"[3PRIME_UTR, STOP_GAINED, FRAME_SHIFT, NON_SYN...",NON_SYNONYMOUS,107,10
ConsDetail,248,"[3_prime_UTR, stop_gained,splice, stop_gained,...",missense,102,21
Segway,248,"[R2, F0, R1, TF2, TF0, L0, R4, L1, R3, R0, GE0...",TF0,76,17


In [22]:
# check if there is missing values for RawScore column
datatable_pd['RawScore'].isnull().values.ravel().sum()

0

In [23]:
datatable_pd.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'INFO', 'Type', 'Length', 'isTv',
       'isDerived',
       ...
       'Intron', 'oAA', 'nAA', 'Grantham', 'PolyPhenCat', 'PolyPhenVal',
       'SIFTcat', 'SIFTval', 'RawScore', 'PHRED'],
      dtype='object', length=117)

In [24]:
null_amount = []
for column in datatable_pd.columns:
    null = datatable_pd[column].isnull().values.ravel().sum()
    if null > 247: # columns without any values
        print(column, null)

targetScan 248
motifECount 248
motifEName 248
motifEHIPos 248
motifEScoreChng 248


In [66]:
# convert all objects in datatable_pd to dummies
# training data
categories={} # contains all the levels in those feature columns
# TODO drop CHROM and POS and etc
categorical_feature_names = [col for col in datatable_pd.columns if col not in ['INFO']]
for f in categorical_feature_names:
    datatable_pd[f] = datatable_pd[f].astype('category')
    categories[f] = datatable_pd[f].cat.categories

new_datatable = pd.get_dummies(datatable_pd,columns=categorical_feature_names)
new_datatable['INFO'] = new_datatable['INFO'].astype('category')
new_datatable['INFO'] = new_datatable['INFO'].cat.codes

In [72]:
new_datatable.shape

(248, 5291)

In [70]:
# split train and test datasets
train, test = train_test_split(new_datatable,test_size=0.25)
print(train.shape, test.shape)

(186, 5291) (62, 5291)


In [55]:
# only use CADD score
model_cadd = LogisticRegression()
model_cadd.fit(train['RawScore'].values.reshape(-1,1),train['INFO'])
pred_cadd = model_cadd.predict(test['RawScore'].values.reshape(-1,1))
tn, fp, fn, tp = confusion_matrix(test['INFO'],pred_cadd).ravel()
print('tp fn fp tn\n%d  %d %d  %d'%(tp,fn,fp,tn))

tp fn fp tn
3  10 4  45


In [54]:
# model using SVM with CADD and PHRED
model_svm = SVC()
model_svm.fit(train['RawScore'].values.reshape(-1,1),train['INFO'])
pred_svm = model_svm.predict(test['RawScore'].values.reshape(-1,1))
tn, fp, fn, tp = confusion_matrix(test['INFO'],pred_svm).ravel()
print('tp fn fp tn\n%d  %d %d  %d'%(tp,fn,fp,tn))


# PHRED provides no interesting information than CADD
# use SVM on CADD score provides slightly, if any, better results...

tp fn fp tn
0  13 0  49


In [76]:
# logistic regression models example for the dummies
model = LogisticRegression()
model.fit(train[[col for col in train.columns if col not in ['INFO']]],train['INFO'])
pred = model.predict(test[[col for col in test.columns if col not in ['INFO']]])
tn, fp, fn, tp = confusion_matrix(test['INFO'],pred).ravel()
print('tp fn fp tn\n%d  %d %d  %d'%(tp,fn,fp,tn))

tp fn fp tn
7  9 1  45
